In [2]:
#remember to cite git

import sys, requests
from bs4 import BeautifulSoup

defaults = {
    'request': {
        'token': 'FpOXt73TnxHjdHDeF0kwIFDipNgN7qApJrHg1smrdQSFSrUEsjzWhuoQ7mKglhPi',
        'base_url': 'https://api.genius.com'
    },
    'message': {
        'search_fail': 'The lyrics for this song were not found!',
        'wrong_input': 'Wrong number of arguments.\n' \
                       'Use two parameters to perform a custom search ' \
                       'or none to get the song currently playing on Spotify.'
    }
}

def request_song_info(song_title, artist_name):
    base_url = defaults['request']['base_url']
    headers = {'Authorization': 'Bearer ' + "FpOXt73TnxHjdHDeF0kwIFDipNgN7qApJrHg1smrdQSFSrUEsjzWhuoQ7mKglhPi"}
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)

    return response

def scrap_song_url(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    [h.extract() for h in html('script')]
    lyrics = html.find('div', class_='lyrics').get_text()

    return lyrics

def main(song_title, artist_name):
    response = request_song_info(song_title, artist_name)
    json = response.json()
    remote_song_info = None

    for hit in json['response']['hits']:
        if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
            remote_song_info = hit
            break

    # Extract lyrics from URL if song was found
    if remote_song_info:
        song_url = remote_song_info['result']['url']
        lyrics = scrap_song_url(song_url)

        #write_lyrics_to_file(lyrics, song_title, artist_name)

        #print(lyrics)
        return lyrics
    else:
        print(defaults['message']['search_fail'])

#consider ways to utilize this
def write_lyrics_to_file (lyrics, song, artist):
    f = open('lyric-view.txt', 'w')
    f.write('{} by {}'.format(song, artist))
    f.write(lyrics)
    f.close()

In [3]:
#the code in this cell was provided by Ulf

import re
import os
import nltk

# Get a list of stopwords from nltk
stopwords = nltk.corpus.stopwords.words("english")

def get_clean_words(wordsss):
    def _isnum(w):
        try:
            int(w)
            return True
        except ValueError:
            return False

    # Remove table and external links
    markup_text = re.sub(r'\{\{[\s\S]*?\}\}', '', wordsss)

    # Remove category links
    markup_text = re.sub(r'\[\[Category.+\]\]', '', markup_text)

    # Set words to lowercase and remove them if they are stop words
    words = [w.lower() for w in re.findall('\w+', markup_text) if w.lower() not in stopwords]

    # Remove numbers
    words = [w for w in words if not _isnum(w)]
    
    word_string = ""
    for word in words:
        word_string += word + " "
        

    return word_string

In [4]:
from matplotlib import pyplot as plt                         #import matplotlib.pyplot
from afinn import Afinn                                      #import Afinn

def get_sentiment_score(words):
            text = get_clean_words(words)        #set text to the string produced by get_clean_words
            word_list = text.split()                         #split the string into a list of words
            num_words = len(word_list)                       #get the number of words in the list
            afinn = Afinn()                                  #initiate Afinn
            score = afinn.score(text)                        #find the Afinn score for the text string
            avg = score/num_words                            #find the average Afinn score per word
            return avg

In [14]:
def sentiment(title, artist):
    lyrics = main(title, artist)
    print lyrics
    score = get_sentiment_score(lyrics)
    return score

sentiment("Without Me", "Halsey")



[Verse 1]
Found you when your heart was broke
I filled your cup until it overflowed
Took it so far to keep you close (Keep you close)
I was afraid to leave you on your own

[Pre-Chorus]
I said I'd catch you if you fall (Fall)
And if they laugh, then fuck 'em all (All)
And then I got you off your knees
Put you right back on your feet
Just so you could take advantage of me

[Chorus]
Tell me, how's it feel sittin' up there?
Feeling so high, but too far away to hold me
You know I'm the one who put you up there
Name in the sky, does it ever get lonely?
Thinking you could live without me
Thinking you could live without me
Baby, I'm the one who put you up there
I don't know why (Yeah, I don't know why)
Thinking you could live without me
Live without me
Baby, I'm the one who put you up there
I don't know why (I don't know why, yeah, yeah)

[Verse 2]
Gave love 'bout a hundred tries (Hundred tries)
Just running from the demons in your mind
Then I took yours and made 'em mine (Mine)
I didn't no

0.009950248756218905